# Chapter 9 Inferences for Proportions and Count Data

In [1]:
import polars as pl
from polars import col, lit
from scipy import stats
import numpy as np

RNG = np.random.default_rng()

## 9.1 Inferences on Proportion

This chapter begins with inference procedures for an unknown proportion $p$ in a Bernoulli population. The sample proportion $\hat{p}$ from a random sample of size $n$ is an unbiased estimate of $p$. Inferences on p are based on the central limit theorem (CLT) result that for large $n$, the sample proportion $\hat{p}$ is approximately normal with mean = $p$ and standard deviation = $\sqrt{pq/n}$ . A large sample two-sided 100(1- $\alpha$)% confidence interval for $p$ is given by

$$
\left[ \hat{p} \pm z_{\alpha /2} \sqrt{\frac{\hat{p} \hat{q}}{n}}\;\right]
$$

where $\hat{q}$ = 1 - $\hat{p}$ and $z_{\alpha/2}$ is the upper $\alpha/2$ critical point of the standard normal distribution. A large sample test on $p$ to test $H_0: p = p_0$ can be based on the test statistic

$$
z = \frac{\hat{p} - p_0}{\sqrt{\hat{p}\hat{q}/n}} \quad \text{or} \quad 
z = \frac{\hat{p} - p_0}{\sqrt{p_0 q_0 / n}}.
$$

Both these statistics are asymptotically standard normal under $H_0$.

### Ex 10.1

Tell whether the following mathematical models are theoretical and deterministic or empirical and probabilistic.

1. Maxwell's equations of electromagnetism. ✍️ theoretical / deterministic
2. An econometric model of the U.S. economy. ✍️ empirical / probabilistic
3. A credit scoring model for the probability of a credit applicant being a good risk as a function of selected variables, e.g., income, outstanding debts, etc. ✍️ empirical / probabilistic

### Ex 10.2

Tell whether the following mathematical models are theoretical and deterministic or empirical and probabilistic.

1. An item response model for the probability of a correct response to an item on a "true-false" test as a function of the item's intrinsic difficulty.  ✍️ empirical / probabilistic
2. The Cobb-Douglas production function, which relates the output of a firm to its capital and labor inputs. ✍️ empirical / probabilistic
3. Kepler's laws of planetary motion. ✍️ theoretical / deterministic

### Ex 10.3

Give an example of an experimental study in which the explanatory variable is controlled at fixed values, while the response variable is random. Also, give an example of an observational study in which both variables are uncontrolled and random.

## 9.2 Inferences for Comparing Two Proportions

Next we consider the problem of comparing two Bernoulli proportions, $p_1$ and $p_2$, based on two independent random samples of sizes $n_1$ and $n_2$. The basis for inferences on $p_1 - p_2$ is the result that for large $n_1$ and $n_2$, the difference in the sample proportions, $\hat{p}_1 - \hat{p}_2$, is approximately normal with mean = $p_1 - p_2$ and standard deviation = $\sqrt{p_1 q_1 / n_1 + p_2 q_2 / n_2}$ . A large sample two- sided 100(1 - $\alpha$)% confidence interval for $p_1 - p_2$ is given by

$$
\left[ \hat{p}_1 - \hat{p}_2 \pm z_{\alpha/2} \sqrt{\frac{\hat{p}_1 \hat{q}_1}{n_1} + \frac{\hat{p}_2 \hat{q}_2}{n_2}}\; \right].
$$

A large sample two-sided $z$-test can be used to test $H_0: p_1 = p_2$ vs. $H_1: p_1 \ne  p_2$ by using the test statistic

$$
z = \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{\frac{\hat{p}_1 \hat{q}_1}{n_1} + \frac{\hat{p}_2 \hat{q}_2}{n_2}}}
\quad \text{or} \quad 
z = \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{\hat{p}\hat{q}\left(\frac{1}{n_1} + \frac{1}{n_2}\right)}}
$$


where $\hat{p} = (n_1\hat{p}_1 + n_2\hat{p}_2)/(n_1 + n_2)$ is the pooled sample proportion. Small sample tests to compare $p_1$ and $p_2$ are also given for independent samples (Fisher's exact test) and matched pairs designs (McNemar's test).

## 9.3 Inferences for One-way Count Data

A generalization of the test on the binomial proportion $p$ is a test on the cell probabilities of a multinomial distribution. Based on a random sample of size $n$ from a $c$-cell multinomial distribution (one-way count data) with cell probabilities $p_1, p_2, \ldots, p_c$, the test of

$$
H_0: p_1 = p_{10},\, p_2 = p_{20},\, \ldots, \, p_c = p_{c0} \quad \text{vs.} \quad
H_1: \text{At least one}\, p_i \ne p_{i0}
$$

is based on the **chi-square statistic** having the general form:

$$
\chi^2 = \sum \frac{(\text{observed} - \text{expected})^2}{\text{expected}}
$$

where "observed" refers to the observed cell counts $n_i$ and "expected" refers to the expected cell counts $e_i = n p_{i0}$ under $H_0$. The degrees of freedom (d.f.) of the chi-square statistic are $c$ - 1. The primary use of this statistic is for the **goodness of fit** test of a specified distribution to a set of data. If any parameters of the distribution are estimated from the data, then one d.f. is deducted for each independent estimated parameter from the total d.f. $c$ - 1.

## 9.4 Inferences for Two-way Count Data

Two-way count data result when

1. a single sample is cross-classified based on two categorical variables into $r$ rows and $c$ columns (**multinomial sampling**), or 
2. independent samples are drawn from $r$ multinomial distributions with the same $c$ categories
(**product multinomial sampling**). 

In both cases, the data are summarized in the form of an $r \times c$ **contingency table** of counts. In case (1), the null hypothesis of interest is the **independence hypothesis** between the row and column variables; in case (2), it is the **homogeneity hypothesis**. In both cases, the chi-square statistic has the same general form given above, with the expected count for the $(i, j)$th cell (under $H_0$) being the $i$th row total times the proportion of all observations falling in the $j$ th column. The d.f. of the chi-square statistic equal $(r - 1)(c - 1)$. Thus association between the row and the column variable is demonstrated at level $\alpha$ if $\chi^2 > \chi^2_{(r-1)(c-1), \alpha}$·